# IPyGPULogger Demo

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import torch

In [3]:
def consume_gpu(n): return torch.ones((n, n)).cuda()
def consume_cpu(n): return np.ones((n, n))

In [4]:
from ipygpulogger import IPyGPULogger
il = IPyGPULogger().start()

RAM: Consumed Peaked  Used Total in 0.000s (In [4])
Gen:        1      0      171 MB
GPU:        0      0     1465 MB


In [5]:
a = consume_cpu(2**12)

RAM: Consumed Peaked  Used Total in 0.033s (In [5])
Gen:      129      0      300 MB
GPU:        0      0     1465 MB


In [6]:
b = consume_gpu(1000)

RAM: Consumed Peaked  Used Total in 1.490s (In [6])
Gen:     1976      0     2276 MB
GPU:      497      0     1962 MB


In [7]:
data = il.data
data
data.gen_mem_used

Data(gen_mem_used_delta=1976, gen_mem_peaked=0, gen_mem_used=2276, gpu_mem_used_delta=497, gpu_mem_peaked=0, gpu_mem_used=1962, time_delta=1.4895367622375488)

2276

RAM: Consumed Peaked  Used Total in 0.005s (In [7])
Gen:        0      0     2276 MB
GPU:        0      0     1962 MB


In [8]:
il.stop()

In [9]:
a = consume_cpu(1000)

In [10]:
%%javascript # prevent committing an unsaved notebook
_=IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>